# Intro to ML: Preprocessing Data

## When and Why Preprocess?
- if data requires cleaning/standardization
- better model performance (in some cases)
- faster training time (in some cases)

## Scaling
- When features have significantly different scales (e.g. min/max scaling)
- Certain algorithms perform better/converge faster if the data is scaled during preprocessing

## Encoding
- Convert categorical variables into numeric values
- example: category values ["low", "medium", "high"] become [1,2,3]

## Principal Component Analysis
- Distill high dimensional data into a lower dimensions